In [60]:
import pandas as pd, numpy as np
reader = pd.read_csv('train.csv',header=None)
feat = reader.values.tolist()
del feat[0]
survived = [float(i[1]) for i in feat]
for i in feat:
    del i[0],i[0],i[1],i[5],i[6],i[6]
for i in feat:
    if i[1] == 'male':
        i[1] = 1
    if i[1] == 'female':
        i[1] = 2
for i in range(891):
    for j in range(6):
        feat[i][j] = float(feat[i][j])
mean=n=[0,0,0,0,0,0]
for i in feat:
    for j in range(6):
        if math.isnan(i[j])==0:
            mean[j]+=i[j]
            n[j]=n[j]+1
mean = [mean[i]/n[i] for i in range(6)]
for i in feat:
    for j in range(6):
        if math.isnan(i[j]):
            i[j] = mean[j]
feat = normalize(np.array(feat).reshape(891,6))
survived = np.array(survived).reshape(891,1)
weights = np.array([0.5 for i in range(6)]).reshape(6,1)
for i in range(500):
    weights = update1(feat,survived,weights,6.3)

In [61]:
reading = pd.read_csv('test.csv',header=None)
fea = reading.values.tolist()
del fea[0]
for i in fea:
    del i[10],i[9],i[7],i[2],i[0]
for i in fea:
    if i[1] == 'male':
        i[1] = 1
    if i[1] == 'female':
        i[1] = 2
for i in range(418):
    for j in range(6):
        fea[i][j] = float(fea[i][j])
mean=n=[0,0,0,0,0,0]
for i in fea:
    for j in range(6):
        if math.isnan(i[j])==0:
            mean[j]+=i[j]
            n[j]=n[j]+1
mean = [mean[i]/n[i] for i in range(6)]
for i in fea:
    for j in range(6):
        if math.isnan(i[j]):
            i[j] = mean[j]
fea = np.array(fea).reshape(418,6)
fea = normalize(fea)
new = np.zeros((418,1))
for i in range(418):
    if predictions(fea,weights)[i]>=0.5:
        new[i][0] = 1
    else:
        new[i][0] = 0
rea = pd.read_csv('gender_submission.csv',header=None)
actual = rea.values.tolist()
del actual[0]
for i in actual:
    del i[0]
    i[0] = float(i[0])
actual = np.array(actual).reshape(418,1)

In [62]:
tp=tn=fp=fn=0
for i in range(418):
    if new[i][0]==actual[i][0]==0.0:
        tn=tn+1
    if new[i][0]==actual[i][0]==1.0:
        tp=tp+1
    if (new[i][0]!=actual[i][0])and(new[i][0]==1.0):
        fp=fp+1
    if (new[i][0]!=actual[i][0])and(new[i][0]==0.0):
        fn=fn+1
accuracy = (tn+tp)/418
print(accuracy*100)
confusion = np.array([[tp,fn],[fp,tn]])
print(confusion)

100.0
[[152   0]
 [  0 266]]


In [63]:
def normalize(feat):
    for f in feat.T:
        mean = np.mean(f)
        n = np.amax(f) - np.amin(f)
        f -= mean
        f /= n
    return feat

In [64]:
def predictions(feat,weights):
    predict = np.zeros((891,1))
    predict = np.dot(feat,weights)
    return predict

In [65]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [66]:
def costfunc(feat,weights,survived):
    prediction = [sigmoid(z) for z in predictions(feat,weights)]
    cost1 = np.zeros((891,1))
    cost2 = np.zeros((891,1))
    for i in range(891):
        cost1[i] = -survived[i]*np.log(prediction[i])
    for i in range(891):
        cost2[i] = -(1-survived[i])*(np.log(1-prediction[i]))
    cost = np.zeros((891,1))
    cost = cost1 + cost2
    cost = cost.sum()/891
    return cost

In [67]:
def update1(feat,survived,weights,lr):
    prediction = predictions(feat,weights)
    for i in range(6):
        for j in range(891):
            a = 1-(2*survived[j][0])
            b = 1 + math.exp(prediction[j][0])
            c = a/b
            x = feat[j,i]
            gradient = x*c*lr/891
        weights[i][0]-=gradient
    return weights

In [68]:
def update2(feat,survived,weights,lr):
    prediction = predictions(feat,weights)
    for i in range(6):
        for j in range(891):
            hx = sigmoid(prediction[j][0])
            y = survived[j][0]
            a = y - hx
            x = feat[j,i]
            gradient = x*a*lr/891
        weights[i][0]+=gradient
    return weights